In [1]:
import scanpy as sc
import cyclum.tuning
import cyclum.models
import copy
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import keras
import keras.backend
from cyclum import writer
import numpy
import numpy as np
import pandas as pd
import seaborn as sns
from data_gen import *

from sklearn.preprocessing import normalize


In [2]:
def get_perm(n):
    perm = np.random.permutation(n)
    E = np.zeros((n,n))
    for i in range(n):
        E[i,perm[i]]=1
    return E , perm

def Perm_to_range(E):
    '''
    :param E: Permutation matrix
    :return: Permutation list
    '''
    order =[]
    for i in range(E.shape[0]):
        for j in range(E.shape[1]):
            if E[i,j]==1:
                order.append(j)
    return np.array(order)
def cell_normalization(V):
    V = normalize(V, axis=1, norm='l2')
    return V

def gene_normalization(V):
    V = normalize(V, axis=0, norm='l2')
    return V

def simulate_gaussian_cyclic(ngenes, ncells):
    A = np.zeros((ncells, ngenes))
    for i in range(ngenes):
        var = int(np.floor(random.rand() * ncells)) + 1
        gene = gaussian(np.arange(ncells),int(ncells/2),var)
        mean = int(np.floor(random.rand() * ncells))
        gene = np.roll(gene,mean)
        A[:,i]=gene
    return A
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))


In [3]:
def cyclum_for_simulated():
    mse_list_linear1 = []
    mse_list_cyclic1 = []
    mse_list_en1 = []
    mse_list_filtered1 = []

    for i in range(100):
        print("Iteration number: " + str(i))
        B = simulate_window_linear(ngenes=500, ncells=200, w=0.2)  # Create linear signal
        ##Normalization improove Cyclum performance
        B = gene_normalization(B)
        E, perm = get_perm(B.shape[0])
        B = E.dot(B)
        perm = Perm_to_range(E)
        A = simulate_spatial_cyclic(ngenes=500, ncells=B.shape[0], w=0.2)  # Create cyclic signal
        A = gene_normalization(A)
        noise = np.random.normal(0, 1e-2, A.shape)
        C = A + B
        C = C + noise
        C = np.clip(C, a_max=np.inf, a_min=0)
        model = cyclum.tuning.CyclumAutoTune(C, max_linear_dims=11,
                                             epochs=300, rate=2e-4, verbose=100,
                                             encoder_width=[20, 30])  # 30,20
        model.train(C, epochs=800, verbose=100, rate=2e-4)
        pseudotime = model.predict_pseudotime(C)
        D = model.get_circular_component(pseudotime)
        K = C - D
        K = np.clip(K,0,np.inf) #Prevents negative expression - improove the results
        D = np.clip(D,0,np.inf) #Prevents negative expression - improove the results
        mse_list_linear1.append(np.linalg.norm(C - B))  # Pre Filtering MSE
        mse_list_cyclic1.append(np.linalg.norm(C - A))  # Pre enhancement MSE
        mse_list_en1.append(np.linalg.norm(D - A))  # Post enhancement MSE
        mse_list_filtered1.append(np.linalg.norm(K - B))  # Post Filtering MSE
    
    with open('mse_list_en_cyclum.npy', 'wb') as f:
        np.save(f, np.array(mse_list_en1))
    with open('mse_list_filtered_cyclum.npy', 'wb') as f:
        np.save(f, np.array(mse_list_filtered1))
    return mse_list_linear1, mse_list_cyclic1 , mse_list_en1 , mse_list_filtered1
A = cyclum_for_simulated()

Iteration number: 0
Auto tuning number of linear components...
Performing PCA...
Training Autoencoder with...
    0 linear dimensions...


2022-11-24 20:21:51.639891: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


epoch: 100/300, loss:  0.0084, time elapsed:  2.21s, time left:  4.41s
epoch: 200/300, loss:  0.0073, time elapsed:  3.19s, time left:  1.59s
epoch: 300/300, loss:  0.0071, time elapsed:  4.17s, time left:  0.00s
[0.0071094101294875145]
[0.010842769571537155, 0.009905248499857064, 0.0091254090475856, 0.008437801922900527, 0.007839554066342631, 0.00731455449607183, 0.006810969177349358, 0.006364016820638799, 0.006019916877532113, 0.00577681964086302, 0.005545123245087278, 0.005339093840262324, 0.0051990534270507675]
    1 linear dimensions...
epoch: 100/300, loss:  0.0079, time elapsed:  1.48s, time left:  2.97s
epoch: 200/300, loss:  0.0068, time elapsed:  2.81s, time left:  1.41s
epoch: 300/300, loss:  0.0064, time elapsed:  4.05s, time left:  0.00s
[0.0071094101294875145, 0.006373073440045118]
[0.010842769571537155, 0.009905248499857064, 0.0091254090475856, 0.008437801922900527, 0.007839554066342631, 0.00731455449607183, 0.006810969177349358, 0.006364016820638799, 0.00601991687753211

epoch: 300/800, loss:  0.0020, time elapsed:  3.81s, time left:  6.35s
epoch: 400/800, loss:  0.0018, time elapsed:  5.06s, time left:  5.06s
epoch: 500/800, loss:  0.0017, time elapsed:  6.12s, time left:  3.67s
epoch: 600/800, loss:  0.0016, time elapsed:  7.17s, time left:  2.39s
epoch: 700/800, loss:  0.0016, time elapsed:  8.20s, time left:  1.17s
epoch: 800/800, loss:  0.0016, time elapsed:  9.23s, time left:  0.00s
Iteration number: 1
Auto tuning number of linear components...
Performing PCA...
Training Autoencoder with...
    0 linear dimensions...
epoch: 100/300, loss:  0.0083, time elapsed:  1.39s, time left:  2.78s
epoch: 200/300, loss:  0.0074, time elapsed:  2.33s, time left:  1.17s
epoch: 300/300, loss:  0.0072, time elapsed:  3.30s, time left:  0.00s
[0.0071609835140407085]
[0.010912524881820674, 0.0099811669412316, 0.009204518338167886, 0.008505563284861972, 0.007897100031896752, 0.007325772571729534, 0.006843356296402325, 0.006391574145280009, 0.0060336714114351975, 0.

epoch: 100/800, loss:  0.0038, time elapsed:  4.26s, time left:  29.79s
epoch: 200/800, loss:  0.0024, time elapsed:  6.40s, time left:  19.21s
epoch: 300/800, loss:  0.0020, time elapsed:  7.90s, time left:  13.17s
epoch: 400/800, loss:  0.0018, time elapsed:  9.79s, time left:  9.79s
epoch: 500/800, loss:  0.0017, time elapsed:  11.91s, time left:  7.14s
epoch: 600/800, loss:  0.0016, time elapsed:  14.05s, time left:  4.68s
epoch: 700/800, loss:  0.0016, time elapsed:  17.08s, time left:  2.44s
epoch: 800/800, loss:  0.0016, time elapsed:  19.91s, time left:  0.00s
Iteration number: 2
Auto tuning number of linear components...
Performing PCA...
Training Autoencoder with...
    0 linear dimensions...
epoch: 100/300, loss:  0.0084, time elapsed:  2.01s, time left:  4.02s
epoch: 200/300, loss:  0.0073, time elapsed:  3.73s, time left:  1.86s
epoch: 300/300, loss:  0.0071, time elapsed:  5.21s, time left:  0.00s
[0.0071259718388319016]
[0.01089456295661855, 0.009977798457000137, 0.00919

...

epoch: 300/300, loss:  0.0019, time elapsed:  3.80s, time left:  0.00s
[0.007024394813925028, 0.006416039075702429, 0.005648321006447077, 0.005035743582993746, 0.004340202547609806, 0.0038542619440704584, 0.0032264823094010353, 0.003066228237003088, 0.002851224271580577, 0.0023952601477503777, 0.0021773020271211863, 0.0019340993603691459]
[0.010803386524086382, 0.009932187212796624, 0.009226373658774519, 0.008554846300028457, 0.007908695017000191, 0.0073272626441834915, 0.006815853496720982, 0.006409501558176317, 0.006063224707711387, 0.0057984007943644705, 0.005560701038199588, 0.005342415144396265, 0.005192003485160458]
epoch: 100/800, loss:  0.0049, time elapsed:  1.70s, time left:  11.88s
epoch: 200/800, loss:  0.0036, time elapsed:  2.85s, time left:  8.56s
epoch: 300/800, loss:  0.0033, time elapsed:  4.01s, time left:  6.69s
epoch: 400/800, loss:  0.0031, time elapsed:  5.17s, time left:  5.17s
epoch: 500/800, loss:  0.0031, time elapsed:  6.42s, time left:  3.85s
epoch: 600/800

epoch: 100/300, loss:  0.0037, time elapsed:  1.85s, time left:  3.71s
epoch: 200/300, loss:  0.0023, time elapsed:  3.10s, time left:  1.55s
epoch: 300/300, loss:  0.0020, time elapsed:  4.34s, time left:  0.00s
[0.007161940447986126, 0.006255211308598518, 0.005514143034815788, 0.0049111004918813705, 0.00444097351282835, 0.003850254463031888, 0.0034618452191352844, 0.0029553731437772512, 0.002611013362184167, 0.002286730334162712, 0.0021756086498498917, 0.0019864977803081274]
[0.010836933039327585, 0.00997988540242316, 0.009203367692095881, 0.00850142118573121, 0.007912513557893219, 0.007341208020410317, 0.006807040728606994, 0.006356884695686639, 0.006008524014246057, 0.005762933488292316, 0.005538314779563499, 0.005318412750949461, 0.0051803875195272015]
Have not found one. Suggest raise max_linear_dims. Use 11 linear components...
epoch: 100/800, loss:  0.0040, time elapsed:  1.77s, time left:  12.41s
epoch: 200/800, loss:  0.0024, time elapsed:  2.94s, time left:  8.81s
epoch: 300

epoch: 100/300, loss:  0.0040, time elapsed:  1.75s, time left:  3.50s
epoch: 200/300, loss:  0.0025, time elapsed:  2.91s, time left:  1.46s
epoch: 300/300, loss:  0.0021, time elapsed:  4.07s, time left:  0.00s
[0.0071777356788516045, 0.006467572413384914, 0.0057375566102564335, 0.005031404551118612, 0.004398260731250048, 0.003834943985566497, 0.0033694168087095022, 0.002966866595670581, 0.0025960528291761875, 0.002460170304402709, 0.0021862804424017668, 0.0020572731737047434]
[0.010859719880515121, 0.009940352831239636, 0.009182321473978244, 0.008474322708570153, 0.007889058338743314, 0.007326636636974291, 0.006803963701248113, 0.0063605622038004, 0.006040099411351044, 0.005761129774775504, 0.005537794278067844, 0.005337735037102654, 0.005179723966129631]
Have not found one. Suggest raise max_linear_dims. Use 11 linear components...
epoch: 100/800, loss:  0.0040, time elapsed:  1.76s, time left:  12.29s
epoch: 200/800, loss:  0.0024, time elapsed:  2.93s, time left:  8.78s
epoch: 30

epoch: 300/300, loss:  0.0023, time elapsed:  3.92s, time left:  0.00s
[0.007224636152386665, 0.0065634953789412975, 0.00559766311198473, 0.0048502348363399506, 0.0044069611467421055, 0.003860162803903222, 0.0033830266911536455, 0.0030456827953457832, 0.0026326647493988276, 0.002427855972200632, 0.00227276305668056]
[0.010949717495648641, 0.010025919126566245, 0.009219063472318849, 0.008434715067926509, 0.007815894070307174, 0.007323653552579966, 0.006872523317734247, 0.0064354069307037565, 0.006112769785931458, 0.005839872866605109, 0.005600859384568521, 0.005376738117783353, 0.005226194005118463]
    11 linear dimensions...
epoch: 100/300, loss:  0.0040, time elapsed:  1.70s, time left:  3.40s
epoch: 200/300, loss:  0.0025, time elapsed:  2.81s, time left:  1.41s
epoch: 300/300, loss:  0.0020, time elapsed:  3.94s, time left:  0.00s
[0.007224636152386665, 0.0065634953789412975, 0.00559766311198473, 0.0048502348363399506, 0.0044069611467421055, 0.003860162803903222, 0.0033830266911536

epoch: 100/300, loss:  0.0040, time elapsed:  1.83s, time left:  3.66s
epoch: 200/300, loss:  0.0025, time elapsed:  3.08s, time left:  1.54s
epoch: 300/300, loss:  0.0022, time elapsed:  4.31s, time left:  0.00s
[0.007077580783516169, 0.006529744248837233, 0.005607696250081062, 0.00490384828299284, 0.004307788796722889, 0.0037210602313280106, 0.0032651128713041544, 0.002998640527948737, 0.0026751989498734474, 0.0023511683102697134, 0.0021857961546629667]
[0.010863134479001498, 0.009940580841528672, 0.009183292786891955, 0.008460123844069292, 0.007856488755199394, 0.007285808574019215, 0.006781449241596721, 0.00641774584337009, 0.006069249376146942, 0.0058141407953451905, 0.005571674691866581, 0.005346481260212312, 0.005193865316312371]
    11 linear dimensions...
epoch: 100/300, loss:  0.0038, time elapsed:  1.84s, time left:  3.68s
epoch: 200/300, loss:  0.0024, time elapsed:  3.07s, time left:  1.53s
epoch: 300/300, loss:  0.0020, time elapsed:  4.29s, time left:  0.00s
[0.007077580

In [4]:
A

([22.515281971632383,
  22.522204643374536,
  22.497866965737938,
  22.495897177133273,
  22.52242998093103,
  22.500741678372563,
  22.504166793963748,
  22.500068515053467,
  22.507957083899836,
  22.506912158480997,
  22.50820601151873,
  22.50395767352902,
  22.489751950913643,
  22.51480151212733,
  22.501747037072832,
  22.499905863770007,
  22.51020984463882,
  22.515766536983403,
  22.5071903366957,
  22.51699144818558,
  22.50565109969104,
  22.510793651842356,
  22.519347980304126,
  22.51983914802748,
  22.508487093812104,
  22.502980991932166,
  22.505607070977245,
  22.521657746907124,
  22.498704862042036,
  22.513708571512502,
  22.50623491847149,
  22.50298950469564,
  22.495730275092587,
  22.507593624729292,
  22.51240261516481,
  22.514389341072672,
  22.489204397111365,
  22.51842762075912,
  22.508085443836727,
  22.509195154463672,
  22.52069696808543,
  22.49272864336158,
  22.5138155142888,
  22.530444918274533,
  22.520132382513868,
  22.507088981672506,
  22.5

In [5]:
print(np.mean(A[0]))
print(np.mean(A[1]))
print(np.mean(A[2]))
print(np.mean(A[3]))

22.509622983817316
22.463888604860426
22.719538943171774
16.40256149853086
